## Object Detection Pipeline on UCS using Darknet & YOLO

This notebook focuses on implementing object detection as a Kubeflow pipeline on Cisco UCS by using Darknet which is a open-source neural network framework, YOLO (You Only Look Once) which is a real-time object detection system.

## Clone Cisco Kubeflow starter pack repository

In [1]:
BRANCH_NAME="master" #Provide git branch "master" or "dev"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 317, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 5519 (delta 147), reused 160 (delta 47), pack-reused 5202
Receiving objects: 100% (5519/5519), 41.06 MiB | 48.21 MiB/s, done.
Resolving deltas: 100% (2109/2109), done.


## Install required packages

In [195]:
!pip install kfp pillow --user

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Restart kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import libraries

In [32]:
import os
import json
import time
import yaml
import calendar
import requests
import logging
import numpy as np

#Kubeflow
import kfp
from kfp.aws import use_aws_secret
import kfp.compiler as compiler

#Kubernetes
from kubernetes import client

#Tensorflow
import tensorflow as tf

## Load pipeline components

Declare the paths of respective YAML configuration files of each of the pipeline components, in order to load each component into a variable for pipeline execution. 

In [33]:
path='components/v2/'
component_root_dwn= path+'download/'
component_root_train= path+'tfjob-create/'
component_root_inference= path+'inference/'

download_op = kfp.components.load_component_from_file(os.path.join(component_root_dwn, 'component.yaml'))
#hptuning_op = kfp.components.load_component_from_file(os.path.join(component_root_katib, 'component.yaml'))
tfjob_create_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml'))
inference_op = kfp.components.load_component_from_file(os.path.join(component_root_inference, 'component.yaml'))

## Define volume claim & volume mount for storage during pipeline execution

Persistent volume claim & volume mount is created for the purpose of storing entities such as Dataset, model files, etc, and to share the stored resources between the various components of the pipeline during it's execution. 

In [34]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

## Define pipeline function

In [42]:
gpus=4 # Number of GPUs to run training

def object_detection_pipeline(
    s3_path="s3://darknet-datasets",    # AWS S3 bucket URL. Ex: s3://<bucket-name>/ 
    namespace='kubeflow',               # Namespace on which trained model is to be deployed for prediction
    timestamp="",                       # Current timestamp
    cfg_data="voc.data",                # Config file containing file name specifications of train, test and validate datasets
    weights="yolov3-voc_50000.weights", # Weights which are already pre-trained upto 50000 iterations is used. Therefore,  
                                        # training happens from 50000 iterations upto a limit of max_batches (say 50200) specified 
                                        # in cfg_file. 
   
    classes_file="voc.names"            # File containing the names of object classes (such as person, bus, car,etc)
):
    
    # Download component
    dwn_task = download_op(s3_path=s3_path,
                           cfg_data=cfg_data
                          ).apply(use_aws_secret(secret_name='aws-secret', aws_access_key_id_name='AWS_ACCESS_KEY_ID', aws_secret_access_key_name='AWS_SECRET_ACCESS_KEY'))
    dwn_task.add_volume(nfs_volume)
    dwn_task.add_volume_mount(nfs_volume_mount) 
    
#     # HP tuning (Katib) component
#     hptuning_task = hptuning_op(cfg_data=cfg_data,             # Config file containing file name specifications of train, test and validate datasets
#                                 cfg_file=cfg_file,             # Config file containing hyperparameters declarations Ex: yolov3.cfg / yolov4.cfg
#                                 weights=weights,               # Pre-trained weights for VOC dataset
#                                 trials=trials,                 # total number of trials under Katib experiment
#                                 timestamp=timestamp,           # Current timestamp to create unique experiment 
#                                                                # Ex: object-detection-1599547688-random-588d7877f5-zvlx5
#                                 gpus_per_trial=gpus_per_trial, # Maximum GPUS to be used for each trial 
#                                 )
#     hptuning_task.add_volume(nfs_volume)
#     hptuning_task.add_volume_mount(nfs_volume_mount)
#     hptuning_task.after(dwn_task)
    
    # TF-job component
    
    tfjob_create_task = tfjob_create_op(timestamp=timestamp)
    tfjob_create_task.add_volume(nfs_volume)
    tfjob_create_task.add_volume_mount(nfs_volume_mount)
    tfjob_create_task.after(dwn_task)
    
    # Inference component
    
    inference_task = inference_op()
    inference_task.add_volume(nfs_volume)
    inference_task.add_volume_mount(nfs_volume_mount)
    inference_task.after(tfjob_create_task)

## Compile pipeline function

Compile the pipeline function to create a tar ball for the pipeline.

In [43]:
# Compile pipeline
try:
    compiler.Compiler().compile(object_detection_pipeline, 'object-detection.tar.gz')
except RuntimeError as err:
    logging.debug(err)
    logging.info("Argo workflow failed validation check but it can still be used to run experiments.")

## Create pipeline experiment

In [44]:
kp_client = kfp.Client()
EXPERIMENT_NAME = 'Object Detection'
experiment = kp_client.create_experiment(name=EXPERIMENT_NAME)

## Initialize pipeline parameters & run pipeline

In [38]:
#Pipeline parameters
timestamp = str(calendar.timegm(time.gmtime()))

# Execute pipeline
run = kp_client.run_pipeline(experiment.id, 'object-detection', 'object-detection.tar.gz', 
                          params={"timestamp": timestamp})